In [ ]:
import pandas as pd
from pathlib import Path

# Read the equipment classes reference file
equipment_classes_path = Path('../data/raw/Equipment Classes.xlsx')
equipment_classes_df = pd.read_excel(equipment_classes_path)

print("Equipment Classes Reference:")
display(equipment_classes_df.head())

In [ ]:
class RefinedEquipmentClassifier:
    def __init__(self, equipment_classes_df):
        """
        Initialize classifier with equipment classes reference data
        """
        self.equipment_classes = {}
        
        # Process equipment classes
        for _, row in equipment_classes_df.iterrows():
            equipment_class = row['Class']
            description = str(row['Description']).lower()
            
            # Create patterns from description
            # Split description into keywords and create patterns
            keywords = description.split()
            
            if equipment_class not in self.equipment_classes:
                self.equipment_classes[equipment_class] = []
                
            # Add the full description as a pattern
            self.equipment_classes[equipment_class].append(
                re.compile(r'\b' + re.escape(description) + r'\b', re.IGNORECASE)
            )
            
            # Add individual keywords and common variations
            for keyword in keywords:
                if len(keyword) > 3:  # Only use keywords longer than 3 characters
                    self.equipment_classes[equipment_class].append(
                        re.compile(r'\b' + re.escape(keyword) + r'\w*\b', re.IGNORECASE)
                    )
    
    def classify_equipment(self, evt_desc, obj_desc=None):
        """
        Classify equipment based primarily on EVT_DESC, using OBJ_DESC as fallback
        """
        # First try matching with EVT_DESC
        evt_desc = str(evt_desc).lower()
        matches = []
        
        # Check each equipment class
        for equipment_class, patterns in self.equipment_classes.items():
            for pattern in patterns:
                if pattern.search(evt_desc):
                    matches.append(equipment_class)
                    break  # Break after first match for this class
        
        # If no matches found in EVT_DESC and OBJ_DESC is provided, try OBJ_DESC
        if not matches and obj_desc:
            obj_desc = str(obj_desc).lower()
            for equipment_class, patterns in self.equipment_classes.items():
                for pattern in patterns:
                    if pattern.search(obj_desc):
                        matches.append(equipment_class)
                        break  # Break after first match for this class
        
        # Return the first match found or 'Other' if no matches
        return matches[0] if matches else 'Other'

# Create instance of refined classifier
refined_classifier = RefinedEquipmentClassifier(equipment_classes_df)

# Apply refined classification
df['equipment_refined'] = df.apply(
    lambda row: refined_classifier.classify_equipment(
        row['EVT_DESC'],
        row['OBJ_DESC'] if 'OBJ_DESC' in row else None
    ),
    axis=1
)

# Show distribution of refined classifications
print("Refined Equipment Classification Distribution:")
display(df['equipment_refined'].value_counts())

# Show sample of door-related classifications
print("\nSample of Door Classifications:")
door_samples = df[df['EVT_DESC'].str.contains('door', case=False, na=False)][
    ['EVT_DESC', 'OBJ_DESC', 'equipment_refined']
].sample(n=10)
display(door_samples)

In [ ]:
# Test specific examples
test_cases = [
    "P1742 Terminal 4 Check 3 doors labeled 2269 to self close properly",
    "T2 Door #2397 and Roof Access Door (Fan Room) Install Door Closers",
    "P1741 Terminal 2 Fan Room Door 2269 check 3 self closing doors",
    "T2 Mechanical Fan Room Doors Need Door Closers"
]

print("Testing specific door-related cases:")
for case in test_cases:
    classification = refined_classifier.classify_equipment(case)
    print(f"\nInput: {case}")
    print(f"Classification: {classification}")

In [ ]:
# Generate summary statistics
summary_stats = {
    'Total Work Orders': len(df),
    'Unique Equipment Types': len(df['equipment'].unique()),
    'Date Range': f"{df['date'].min()} to {df['date'].max()}" if 'date' in df.columns else 'N/A',
    'Most Common Equipment': df['equipment'].mode().iloc[0],
    'Equipment Coverage': f"{(df['equipment'] != 'Unknown').mean():.1%}"
}

# Display summary
for key, value in summary_stats.items():
    print(f"{key}: {value}")

# If we have priority or status information, show distribution
if 'priority' in df.columns:
    print("\nPriority Distribution:")
    display(df['priority'].value_counts())

if 'status' in df.columns:
    print("\nStatus Distribution:")
    display(df['status'].value_counts())

## 7. Summary and Insights

Based on our analysis, here are the key findings:

1. Equipment Distribution:
   - List the most common equipment types
   - Identify any patterns in equipment maintenance

2. Temporal Patterns:
   - Peak maintenance times
   - Seasonal trends (if any)
   - Day of week patterns

3. Key Insights:
   - Notable patterns in work order descriptions
   - Potential areas for optimization
   - Recommendations for maintenance scheduling

In [ ]:
# Create a heatmap of work orders by day of week and hour (if date/time data available)
if 'date' in df.columns:
    df['day_of_week'] = df['date'].dt.day_name()
    df['hour'] = df['date'].dt.hour
    
    # Create pivot table for heatmap
    heatmap_data = pd.pivot_table(
        df,
        values='id',
        index='day_of_week',
        columns='hour',
        aggfunc='count',
        fill_value=0
    )
    
    # Create heatmap
    fig = go.Figure(data=go.Heatmap(
        z=heatmap_data.values,
        x=heatmap_data.columns,
        y=heatmap_data.index,
        colorscale='Viridis'
    ))
    
    fig.update_layout(
        title='Work Order Frequency by Day and Hour',
        xaxis_title='Hour of Day',
        yaxis_title='Day of Week'
    )
    fig.show()

# Create word cloud of common terms in descriptions (if wordcloud is installed)
try:
    from wordcloud import WordCloud
    
    # Combine all descriptions
    text = ' '.join(df['description'].fillna(''))
    
    # Generate and display word cloud
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('Common Terms in Work Order Descriptions')
    plt.show()
except ImportError:
    print("WordCloud package not installed. Skip word cloud visualization.")

## 6. Pattern Visualization

Let's create some detailed visualizations to better understand the patterns in our data:

In [ ]:
# Analyze work orders by equipment type
equipment_stats = df.groupby('equipment').agg({
    'id': 'count',  # number of work orders
}).reset_index()

equipment_stats = equipment_stats.rename(columns={'id': 'work_order_count'})
equipment_stats = equipment_stats.sort_values('work_order_count', ascending=False)

print("Equipment Statistics:")
display(equipment_stats.head(10))

# If we have date information, analyze trends over time
if 'date' in df.columns:
    # Resample by month and count work orders
    monthly_orders = df.set_index('date')['id'].resample('M').count()
    
    # Create time series plot
    fig = px.line(
        monthly_orders, 
        title='Work Orders Over Time',
        labels={'value': 'Number of Work Orders', 'index': 'Date'}
    )
    fig.show()

# Equipment type distribution
fig = px.pie(
    equipment_stats.head(10), 
    values='work_order_count', 
    names='equipment',
    title='Distribution of Top 10 Equipment Types'
)
fig.show()

## 5. Exploratory Data Analysis

Let's analyze patterns and trends in the work orders:

In [9]:
# Check for missing values
print("Missing values in each column:")
display(df.isnull().sum())

# Remove duplicates if any exist
n_before = len(df)
df = df.drop_duplicates()
n_after = len(df)
print(f"\nRemoved {n_before - n_after} duplicate rows")

# Handle missing equipment values
df['equipment'] = df['equipment'].fillna('Unknown')

# Convert date columns to datetime if they exist
date_columns = df.select_dtypes(include=['datetime64']).columns
for col in date_columns:
    df[col] = pd.to_datetime(df[col])
    print(f"\nConverted {col} to datetime")

# Display the cleaned dataset info
print("\nCleaned dataset info:")
print(df.info())

Missing values in each column:


EVT_ORG                          0
EVT_CODE                         0
EVT_OBJECT                       0
OBJ_CODE                         5
OBJ_DESC                         5
OBJ_CLASS                    63349
OBJ_CATEGORY                 75846
EVT_DESC                         0
EVT_STATUS                       0
Status                           0
EVT_PRIORITY                 31698
Priority                         0
EVT_TYPE                         0
EVT_JOBTYPE                      0
WO Type                          0
EVT_REQM                      2003
EVT_CAUSE                     2579
EVT_CREATEDBY                    0
EVT_DATE                         0
EVT_COMPLETED                  263
EVO_MATERIAL                    18
EVO_TOTAL                       18
EVO_HOURS                       18
Est Labor Cost                  18
CALL TIME                    43511
ARRIVAL TIME                 42523
COMPLETED TIME               42669
RESPONSE TIME - SECONDS      46280
RESPONSE TIME - MINU


Removed 0 duplicate rows

Converted EVT_DATE to datetime

Converted EVT_COMPLETED to datetime

Converted CALL TIME to datetime

Converted ARRIVAL TIME to datetime

Converted COMPLETED TIME to datetime

Cleaned dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81131 entries, 0 to 81130
Data columns (total 32 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   EVT_ORG                    81131 non-null  object        
 1   EVT_CODE                   81131 non-null  int64         
 2   EVT_OBJECT                 81131 non-null  object        
 3   OBJ_CODE                   81126 non-null  object        
 4   OBJ_DESC                   81126 non-null  object        
 5   OBJ_CLASS                  17782 non-null  object        
 6   OBJ_CATEGORY               5285 non-null   object        
 7   EVT_DESC                   81131 non-null  object        
 8   EVT_STATUS                 8113

## 4. Data Cleaning and Preprocessing

Let's clean the data and handle any missing values or inconsistencies:

In [8]:
# Initialize the equipment extractor
equipment_extractor = EquipmentExtractor()

# Extract equipment from descriptions and object descriptions
df['equipment'] = df.apply(
    lambda row: equipment_extractor.extract_type(
        description=row['EVT_DESC'],
        obj_desc=row['OBJ_DESC']
    ),
    axis=1
)

# Count the frequency of each equipment type
equipment_counts = df['equipment'].value_counts()

# Display the top 10 most common equipment types
print("Top 10 Most Common Equipment Types:")
display(equipment_counts.head(10))

# Create a bar chart of equipment frequencies
fig = px.bar(
    equipment_counts.head(20),
    title='Most Common Equipment Types in Work Orders',
    labels={'index': 'Equipment Type', 'value': 'Number of Work Orders'},
    height=500
)
fig.show()

Top 10 Most Common Equipment Types:


equipment
Plumbing               19847
Doors/Windows          19653
Other/Unclassified     18490
Electrical/Lighting     5666
HVAC                    5054
Elevator/Escalator      2405
Flooring                2215
Roofing                 1661
Parking                 1657
Fire Safety             1509
Name: count, dtype: int64

In [10]:
# Create a comparison DataFrame with relevant columns
comparison_df = df[[
    'EVT_DESC',
    'OBJ_DESC',
    'OBJ_CLASS',
    'OBJ_CATEGORY',
    'equipment'
]].copy()

# Export to Excel
output_path = Path('../data/raw/equipment_classification_results.xlsx')
comparison_df.to_excel(output_path, index=False)

print(f"Exported comparison data to: {output_path}")

# Display a sample of the results
print("\nSample of classifications:")
display(comparison_df.head(10))

Exported comparison data to: ..\data\raw\equipment_classification_results.xlsx

Sample of classifications:


,EVT_DESC,OBJ_DESC,OBJ_CLASS,OBJ_CATEGORY,equipment
0,T2 Women's Restroom #2535: 1 light out above t...,T2.F2-2535 / 3-202.2-WOMEN'S RESTROOM (ROOM DR...,NaN,NaN,Plumbing
1,Archibald Entrance to Airport By Palm Trees Br...,T2.F1.-CURBSIDE,NaN,NaN,Electrical/Lighting
2,"Maintenance Yard Sign Shop provide 100 ""ONT"" s...",Signage,FAC,NaN,Signage
3,"T2/T4 Provide 100 ""ONT"" Signs to Cover existin...",T4.F1-CHECK-IN AREA,NaN,NaN,Doors/Windows
4,T2 and T4 5 Authorized Parking Signs Re-enfore...,T2.F1.-CURBSIDE,NaN,NaN,Signage
5,OIAA Admin Office: Relocate Items to Office 104,ADM.OFFCE.F1-ADMIN OFFICE,NaN,NaN,Other/Unclassified
6,OIAA Admin Repair for ADA Ramp,ADM.OFFCE.F1-ADMIN OFFICE,NaN,NaN,Structural
7,OIAA Admin Badging Offices Paint Walls,ADM.OFFCE.F1-ADMIN OFFICE,NaN,NaN,Structural
8,"T2 Cargo Elevator #1 Making ""Bip"" Noise. Inspe...","TERMINAL 2, 1ST FLOOR VEST #1205 ELEVATOR #1 L...",EQUIPMEN,NaN,Elevator/Escalator
9,OIAA Admin Building Badge Office: Make a frame...,ADM.OFFCE.F1-ADMIN OFFICE,NaN,NaN,Doors/Windows


## 3. Equipment Pattern Extraction

Now we'll extract equipment information from the work order descriptions using our pattern matching system:

In [4]:
# Initialize the Excel reader
data_path = Path('../data/raw/Ontario Facilities Work Order Data.xlsx')
reader = WorkOrderReader(data_path)

# Load the data
try:
    df = reader.read_work_order_sheets()
    print(f"Successfully loaded data with {len(df)} rows")
except FileNotFoundError:
    print(f"Error: Could not find file at {data_path}")
    print("Please ensure the work orders Excel file is in the correct location")
except Exception as e:
    print(f"Error loading data: {str(e)}")

# Display basic information about the dataset
print("\nDataset Info:")
print(df.info())

# Show the first few rows
print("\nFirst few rows:")
display(df.head())

Successfully loaded data with 81131 rows

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81131 entries, 0 to 81130
Data columns (total 31 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   EVT_ORG                    81131 non-null  object        
 1   EVT_CODE                   81131 non-null  int64         
 2   EVT_OBJECT                 81131 non-null  object        
 3   OBJ_CODE                   81126 non-null  object        
 4   OBJ_DESC                   81126 non-null  object        
 5   OBJ_CLASS                  17782 non-null  object        
 6   OBJ_CATEGORY               5285 non-null   object        
 7   EVT_DESC                   81131 non-null  object        
 8   EVT_STATUS                 81131 non-null  object        
 9   Status                     81131 non-null  object        
 10  EVT_PRIORITY               49433 non-null  object        
 11  Priority   

,EVT_ORG,EVT_CODE,EVT_OBJECT,OBJ_CODE,OBJ_DESC,OBJ_CLASS,OBJ_CATEGORY,EVT_DESC,EVT_STATUS,Status,EVT_PRIORITY,Priority,EVT_TYPE,EVT_JOBTYPE,WO Type,EVT_REQM,EVT_CAUSE,EVT_CREATEDBY,EVT_DATE,EVT_COMPLETED,EVO_MATERIAL,EVO_TOTAL,EVO_HOURS,Est Labor Cost,CALL TIME,ARRIVAL TIME,COMPLETED TIME,RESPONSE TIME - SECONDS,RESPONSE TIME - MINUTES,COMPLETION TIME - MINUTES,Source_Sheet
0,ONT.FAC,2717363,L.T2.F2.10237,L.T2.F2.10237,T2.F2-2535 / 3-202.2-WOMEN'S RESTROOM (ROOM DR...,NaN,NaN,T2 Women's Restroom #2535: 1 light out above t...,C,Completed,6,Pri 6: Scheduled,JOB,RM,Reactive Maintenance,LIGHT-OU,ELEC,CALDERONS,2018-02-26 08:25:00.000,2018-08-15 13:44:59.999,0.0,0.0,2.25,78.75,2018-08-15 06:45:00,2018-08-15 11:29:59.999,2018-08-15 13:44:59.999,-17100.0,-285.0,-135.0,Page1
1,ONT.FAC,2708319,L.T2.F1.10505,L.T2.F1.10505,T2.F1.-CURBSIDE,NaN,NaN,Archibald Entrance to Airport By Palm Trees Br...,C,Completed,6,Pri 6: Scheduled,JOB,RM,Reactive Maintenance,LNW,ELEC,CALDERONS,2018-03-01 09:28:59.999,2018-07-02 08:29:59.999,0.0,0.0,1.00,35.00,2018-07-02 06:45:00,2018-07-02 07:30:00.000,2018-07-02 08:29:59.999,-2700.0,-45.0,-60.0,Page1
2,ONT.FAC,2550249,10716,10716,Signage,FAC,NaN,"Maintenance Yard Sign Shop provide 100 ""ONT"" s...",C,Completed,6,Pri 6: Scheduled,JOB,RM,Reactive Maintenance,ADD-SCOP,COMMIS,SANCHEZC,2018-03-12 19:57:00.000,2018-08-30 12:59:00.000,0.0,0.0,5.00,175.00,2018-08-30 06:18:00,2018-08-30 07:58:00.000,2018-08-30 12:57:59.999,-6000.0,-100.0,-300.0,Page1
3,ONT.FAC,2708573,L.T4.F1.10575,L.T4.F1.10575,T4.F1-CHECK-IN AREA,NaN,NaN,"T2/T4 Provide 100 ""ONT"" Signs to Cover existin...",C,Completed,6,Pri 6: Scheduled,JOB,CHILD CM,Child Cm,ADD-SCOP,COMMIS,CALDERONS,2018-03-13 15:43:00.000,2018-08-30 12:00:00.000,0.0,0.0,5.00,175.00,2018-08-30 06:45:00,2018-08-30 06:59:59.999,2018-08-30 12:00:00.000,-900.0,-15.0,-300.0,Page1
4,ONT.FAC,2552698,L.T2.F1.10505,L.T2.F1.10505,T2.F1.-CURBSIDE,NaN,NaN,T2 and T4 5 Authorized Parking Signs Re-enfore...,C,Completed,6,Pri 6: Scheduled,JOB,RM,Reactive Maintenance,ADD-SCOP,NOPROB,CALDERONS,2018-04-17 09:46:59.999,2018-07-31 12:59:59.999,0.0,0.0,6.00,210.00,2018-07-31 06:45:00,2018-07-31 06:59:59.999,2018-07-31 12:59:59.999,-900.0,-15.0,-360.0,Page1


In [6]:
# Display column names
print("\nColumn names in the dataset:")
print(df.columns.tolist())


Column names in the dataset:
['EVT_ORG', 'EVT_CODE', 'EVT_OBJECT', 'OBJ_CODE', 'OBJ_DESC', 'OBJ_CLASS', 'OBJ_CATEGORY', 'EVT_DESC', 'EVT_STATUS', 'Status', 'EVT_PRIORITY', 'Priority', 'EVT_TYPE', 'EVT_JOBTYPE', 'WO Type', 'EVT_REQM', 'EVT_CAUSE', 'EVT_CREATEDBY', 'EVT_DATE', 'EVT_COMPLETED', 'EVO_MATERIAL', 'EVO_TOTAL', 'EVO_HOURS', 'Est Labor Cost', 'CALL TIME', 'ARRIVAL TIME', 'COMPLETED TIME', 'RESPONSE TIME - SECONDS', 'RESPONSE TIME - MINUTES', 'COMPLETION TIME - MINUTES', 'Source_Sheet']


## 2. Data Loading and Validation

We'll load the maintenance work orders data and perform initial validation checks:

In [3]:
# Standard data analysis libraries
import pandas as pd
import numpy as np

# Visualization libraries
import plotly.express as px
import plotly.graph_objects as go

# System utilities
import os
import sys
from pathlib import Path

# Add the project root to the Python path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Import our custom modules
from src.readers.excel_reader import WorkOrderReader
from src.extractors.equipment import EquipmentExtractor

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

## 1. Setup and Imports

First, we'll import the necessary libraries for our analysis:

# Maintenance Work Orders Analysis

This notebook performs exploratory data analysis on maintenance work orders to identify equipment patterns, maintenance trends, and potential insights for optimization.

## Table of Contents
1. Setup and Imports
2. Data Loading and Validation
3. Equipment Pattern Extraction
4. Data Cleaning and Preprocessing
5. Exploratory Data Analysis
6. Pattern Visualization
7. Summary and Insights

In [11]:
# Analyze Air Handler Unit classifications
air_handler_mask = df['EVT_DESC'].str.contains('Air Handler|AHU', case=False, na=False) | \
                  df['OBJ_DESC'].str.contains('Air Handler|AHU', case=False, na=False)

air_handler_analysis = df[air_handler_mask][['EVT_DESC', 'OBJ_DESC', 'OBJ_CLASS', 'OBJ_CATEGORY', 'equipment']]
print("Sample of Air Handler Unit Classifications:")
display(air_handler_analysis.head(10))

print("\nDistribution of equipment types for Air Handler entries:")
display(air_handler_analysis['equipment'].value_counts())

Sample of Air Handler Unit Classifications:


,EVT_DESC,OBJ_DESC,OBJ_CLASS,OBJ_CATEGORY,equipment
75,T4 Fan Room 1059 Air Handler 4.13 Repairs/Adju...,AHU 4.13,FAC,HVAC,HVAC
77,T4 Fan Room 2397 Air Handler 4.7 and 4.11 Repl...,AHU 4.7,FAC,HVAC,HVAC
121,T1 Troubleshoot Air Handler On Roof,T1.F1.-CHECK-IN AREA,NaN,NaN,Roofing
126,T2 Air Handling Unit Not Operating. Troublesho...,T2.F1-1150 / 2-103.0-FAN ROOM (ROOM 1DR),NaN,NaN,HVAC
141,T4 Room 2397 Replaced Air Filters in Air Handl...,AHU 4.10,FAC,HVAC,Other/Unclassified
210,Terminal 1 Reset Air Handler Breaker and Verify,T1.F1.-CHECK-IN AREA,NaN,NaN,Other/Unclassified
211,Terminal 4 ONTEC Offices No Airflow Reset VFD ...,AHU 4.11,FAC,HVAC,Other/Unclassified
293,T4 AHU 4.10: Found VFD lockout on overload. Re...,AHU 4.10,FAC,HVAC,Doors/Windows
373,T4 Room 2397: Reset VFD for AHU 4.10,AHU 4.10,FAC,HVAC,Other/Unclassified
400,Terminal 4 AHU 4.10 tech found out on overload...,AHU 4.10,FAC,HVAC,Other/Unclassified



Distribution of equipment types for Air Handler entries:


equipment
Other/Unclassified     1528
HVAC                    180
Plumbing                112
Flooring                 89
Baggage Handling         64
Electrical/Lighting      44
Doors/Windows            14
Roofing                   7
Elevator/Escalator        5
Structural                3
Parking                   1
Signage                   1
Fire Safety               1
Name: count, dtype: int64

In [14]:
import re

# Detailed pattern analysis for Air Handler entries
def analyze_description_patterns(text_series):
    # Common patterns we might want to add to our classifier
    patterns = {
        'ahu_number': r'(AHU[- ]?\d+)',
        'air_handler': r'(Air Handler\w*)',
        'detailed_desc': r'((?:filter|belt|motor|fan|cooling|heating|ventilation))'
    }
    
    results = {}
    for pattern_name, pattern in patterns.items():
        matches = text_series.str.extractall(pattern, flags=re.IGNORECASE)
        unique_matches = matches[0].unique() if not matches.empty else []
        results[pattern_name] = unique_matches
    
    return results

print("Patterns in Event Descriptions:")
evt_patterns = analyze_description_patterns(air_handler_analysis['EVT_DESC'])
for pattern, matches in evt_patterns.items():
    print(f"\n{pattern}:")
    print(matches[:10])  # Show first 10 matches

print("\nPatterns in Object Descriptions:")
obj_patterns = analyze_description_patterns(air_handler_analysis['OBJ_DESC'])
for pattern, matches in obj_patterns.items():
    print(f"\n{pattern}:")
    print(matches[:10])  # Show first 10 matches

# Show distribution of object classes for air handlers
print("\nObject Classes for Air Handlers:")
display(air_handler_analysis['OBJ_CLASS'].value_counts())

Patterns in Event Descriptions:

ahu_number:
['AHU 4' 'AHU 2' 'AHU 3' 'ahu 2' 'AHU2' 'ahu 4' 'Ahu 4' 'ahu 3' 'Ahu 2'
 'AHU 410']

air_handler:
['Air Handler' 'air handler' 'air handlers' 'Air handler' 'Air Handlers'
 'AIR HANDLERS' 'AIR HANDLER' 'air Handler']

detailed_desc:
['Fan' 'Filter' 'filter' 'motor' 'belt' 'fan' 'heating' 'Motor' 'Belt'
 'FAN']

Patterns in Object Descriptions:

ahu_number:
['AHU 4' 'AHU 2' 'AHU 3']

air_handler:
['AIR HANDLER']

detailed_desc:
['FAN' 'COOLING']

Object Classes for Air Handlers:


OBJ_CLASS
FAC         1815
EQUIPMEN      20
Name: count, dtype: int64

## HVAC Equipment Sub-Classification

Let's analyze the distribution of specific HVAC equipment types within our HVAC-classified records:

In [17]:
# Create patterns for different HVAC equipment types
hvac_patterns = {
    'RTU': r'(?i)(?:RTU|Roof\s*Top\s*Unit)',
    'AHU': r'(?i)(?:AHU|Air\s*Handler\s*Unit)',
    'Boiler': r'(?i)boiler',
    'Chiller': r'(?i)chiller',
    'Minisplit': r'(?i)(?:mini\s*split|mini-split)',
    'PCA': r'(?i)(?:PCA|Pre[\s-]?Conditioned\s*Air)',
    'VAV': r'(?i)(?:VAV|Variable\s*Air\s*Volume)',
    'Fan Coil': r'(?i)fan\s*coil',
    'Heat Pump': r'(?i)heat\s*pump',
    'Cooling Tower': r'(?i)cooling\s*tower'
}

# Get all HVAC records
hvac_records = df[df['equipment'] == 'HVAC']

# Analyze distribution of specific HVAC types
results = {}
overlapping = 0
multiple_types = []

# First pass - count individual patterns
for equip_type, pattern in hvac_patterns.items():
    mask = (
        hvac_records['EVT_DESC'].str.contains(pattern, regex=True, na=False) |
        hvac_records['OBJ_DESC'].str.contains(pattern, regex=True, na=False)
    )
    results[equip_type] = mask.sum()
    
    # Show a sample of matches
    print(f"\n{equip_type} examples:")
    sample = hvac_records[mask].sample(n=min(3, mask.sum()))[['EVT_DESC', 'OBJ_DESC']]
    display(sample)

# Second pass - check for records with multiple types
for idx, row in hvac_records.iterrows():
    types_found = []
    for equip_type, pattern in hvac_patterns.items():
        if (re.search(pattern, str(row['EVT_DESC']), re.IGNORECASE) or 
            re.search(pattern, str(row['OBJ_DESC']), re.IGNORECASE)):
            types_found.append(equip_type)
    if len(types_found) > 1:
        overlapping += 1
        multiple_types.append(types_found)

print("\nHVAC Equipment Type Distribution:")
for equip_type, count in sorted(results.items(), key=lambda x: x[1], reverse=True):
    print(f"{equip_type}: {count}")

# Calculate unclassified
total_hvac = len(hvac_records)
records_with_type = sum(mask.sum() for equip_type, pattern in hvac_patterns.items())
print(f"\nTotal HVAC records: {total_hvac}")
print(f"Records with at least one type: {records_with_type}")
print(f"Records with multiple types: {overlapping}")
print(f"Records with no specific type: {total_hvac - records_with_type}")

if multiple_types:
    print("\nSample of records with multiple types:")
    for types in multiple_types[:5]:
        print(f"- Found types: {', '.join(types)}")


RTU examples:


,EVT_DESC,OBJ_DESC
45744,FIS A/C #8 DISCONNECT POWER,FIS RTU #8
4530,FIS RTU# 11Condenser Fan Motor & Cap,FIS RTU #11
8586,FIS A/C 21 Replace the indoor blower motor,FIS RTU #21



AHU examples:


,EVT_DESC,OBJ_DESC
13699,T4 fan coil #3 water leak,AHU 4.3
16828,T2 AHU 3.14 replace 2 pulleys and 2 belts ret...,AHU 3.14
52042,Terminal 4 Room 1186 Exhaust Fan 4.12 screen N...,AHU 4.12



Boiler examples:


...


Heat Pump examples:


,EVT_DESC,OBJ_DESC



Cooling Tower examples:


,EVT_DESC,OBJ_DESC
60531,3.4 Cooling Tower Double Fan Blade Assembly,COOLING TOWER 3.4
9825,"T2, COOLING TOWER 2.1, FAN #1 FAILED",COOLING TOWER 2.1 BALTIMORE AIR COIL
18309,T-2 CT 2.1 Fan motor 4 fail,COOLING TOWER 2.1 BALTIMORE AIR COIL



HVAC Equipment Type Distribution:
Fan Coil: 445
AHU: 157
Chiller: 76
RTU: 70
Cooling Tower: 33
Boiler: 4
Minisplit: 2
VAV: 2
PCA: 1
Heat Pump: 0

Total HVAC records: 5054
Records with at least one type: 330
Records with multiple types: 11
Records with no specific type: 4724

Sample of records with multiple types:
- Found types: AHU, Chiller
- Found types: AHU, Fan Coil
- Found types: RTU, Fan Coil
- Found types: RTU, Fan Coil
- Found types: AHU, Fan Coil


In [ ]:
# Enhanced HVAC classification function
def classify_hvac_equipment(evt_desc, obj_desc):
    evt_desc = str(evt_desc).upper()
    obj_desc = str(obj_desc).upper()
    combined_text = f"{evt_desc} {obj_desc}"
    
    # Define patterns with variations
    patterns = {
        'RTU': [
            r'RTU[\s#-]?\d*',
            r'ROOF\s*TOP\s*UNIT',
            r'ROOFTOP\s*UNIT'
        ],
        'AHU': [
            r'AHU[\s#-]?\d*',
            r'AIR\s*HANDLER',
            r'AIR\s*HANDLING\s*UNIT'
        ],
        'Boiler': [
            r'BOILER[\s#-]?\d*',
            r'HEATING\s*SYSTEM'
        ],
        'Chiller': [
            r'CHILLER[\s#-]?\d*',
            r'CHILLED\s*WATER'
        ],
        'Minisplit': [
            r'MINI[\s-]?SPLIT',
            r'DUCTLESS[\s-]?SPLIT'
        ],
        'PCA': [
            r'PCA[\s#-]?\d*',
            r'PRE[\s-]?CONDITIONED\s*AIR'
        ],
        'VAV': [
            r'VAV[\s#-]?\d*',
            r'VARIABLE\s*AIR\s*VOLUME'
        ],
        'Fan Coil': [
            r'FAN\s*COIL',
            r'FCU[\s#-]?\d*'
        ],
        'Heat Pump': [
            r'HEAT\s*PUMP',
            r'HP[\s#-]?\d*'
        ],
        'Cooling Tower': [
            r'COOLING\s*TOWER',
            r'CT[\s#-]?\d*'
        ]
    }
    
    # Check for each type
    matches = []
    for equip_type, pattern_list in patterns.items():
        for pattern in pattern_list:
            if re.search(pattern, combined_text):
                matches.append(equip_type)
                break
    
    if len(matches) == 1:
        return matches[0]
    elif len(matches) > 1:
        return matches[0]  # Return the first match for now
    else:
        # Additional context-based classification
        if any(term in combined_text for term in ['CONDENSER', 'CONDENSING UNIT']):
            return 'RTU'
        elif any(term in combined_text for term in ['HVAC FILTER', 'AIR FILTER']):
            if 'ROOF' in combined_text:
                return 'RTU'
            else:
                return 'AHU'
        return 'Other HVAC'

# Apply the new classification
df['hvac_type'] = df[df['equipment'] == 'HVAC'].apply(
    lambda row: classify_hvac_equipment(row['EVT_DESC'], row['OBJ_DESC']),
    axis=1
)

# Show the distribution of the new classification
print("New HVAC Equipment Type Distribution:")
display(df['hvac_type'].value_counts(dropna=False))

# Show some examples of each type
for hvac_type in df['hvac_type'].unique():
    if pd.isna(hvac_type):
        continue
    print(f"\nExamples of {hvac_type}:")
    sample = df[df['hvac_type'] == hvac_type].sample(n=min(3, len(df[df['hvac_type'] == hvac_type])))[['EVT_DESC', 'OBJ_DESC']]
    display(sample)

## NLP-Based Equipment Classification

Let's implement an NLP-based approach to improve equipment classification across all categories. This will help us better identify specific equipment types from work order descriptions.

In [18]:
# Install required NLP packages
import sys
import subprocess

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

required_packages = [
    'nltk',
    'scikit-learn',
    'gensim'
]

for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        print(f"Installing {package}...")
        install_package(package)

import nltk
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')
try:
    nltk.data.find('tokenizers/averaged_perceptron_tagger')
except LookupError:
    nltk.download('averaged_perceptron_tagger')
try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

Installing nltk...
Installing scikit-learn...
Installing gensim...


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marcr\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marcr\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\marcr\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marcr\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\marcr\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\marcr\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading pack

In [19]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re

class NLPEquipmentClassifier:
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        
        # Define equipment categories and their related terms
        self.equipment_patterns = {
            # HVAC Equipment
            'AHU': ['air handler', 'air handling unit', 'ahu'],
            'RTU': ['roof top unit', 'rooftop unit', 'rtu'],
            'Chiller': ['chiller', 'cooling system', 'chilled water'],
            'Boiler': ['boiler', 'heating system', 'steam system'],
            'PCA': ['pre conditioned air', 'preconditioned air', 'pca'],
            'Minisplit': ['mini split', 'minisplit', 'ductless'],
            'Cooling Tower': ['cooling tower', 'condenser tower'],
            
            # Fire Safety Equipment
            'Fire Panel': ['fire panel', 'fire alarm', 'fire control', 'fire system'],
            'Fire Extinguisher': ['fire extinguisher', 'extinguisher'],
            'Sprinkler': ['sprinkler', 'fire suppression'],
            
            # Plumbing Equipment
            'Eye Wash Station': ['eye wash', 'eyewash'],
            'Water Heater': ['water heater', 'hot water'],
            'Drinking Fountain': ['drinking fountain', 'water fountain'],
            
            # Waste Management
            'Trash Compactor': ['trash compactor', 'compactor', 'waste compactor'],
            'Dumpster': ['dumpster', 'waste container'],
            
            # Building Systems
            'Elevator': ['elevator', 'lift'],
            'Escalator': ['escalator', 'moving stairway'],
            'Door': ['door', 'entrance', 'exit', 'doorway'],
            'Window': ['window', 'glazing'],
            
            # Electrical Systems
            'Electrical Panel': ['electrical panel', 'circuit breaker', 'power panel'],
            'Generator': ['generator', 'power generator', 'emergency power'],
            'UPS': ['ups', 'uninterruptible power', 'battery backup'],
            
            # Baggage Systems
            'Baggage Belt': ['baggage belt', 'conveyor belt', 'luggage conveyor'],
            'Baggage Carousel': ['carousel', 'baggage carousel', 'luggage carousel']
        }
        
        # Compile regex patterns
        self.compiled_patterns = {
            category: [re.compile(r'\b' + term.replace(' ', r'\s+') + r'\b', re.IGNORECASE)
                      for term in terms]
            for category, terms in self.equipment_patterns.items()
        }
        
    def preprocess_text(self, text):
        """Preprocess text by tokenizing, removing stopwords, and lemmatizing"""
        text = str(text).lower()
        tokens = word_tokenize(text)
        tokens = [self.lemmatizer.lemmatize(token) for token in tokens if token not in self.stop_words]
        return ' '.join(tokens)
    
    def classify_equipment(self, evt_desc, obj_desc):
        """Classify equipment based on event and object descriptions"""
        # Combine descriptions
        combined_text = f"{evt_desc} {obj_desc}"
        
        # Check for equipment mentions
        matches = []
        for category, patterns in self.compiled_patterns.items():
            if any(pattern.search(combined_text) for pattern in patterns):
                matches.append(category)
        
        if len(matches) == 1:
            return matches[0]
        elif len(matches) > 1:
            # If multiple matches, use TF-IDF similarity to find best match
            vectorizer = TfidfVectorizer()
            category_vectors = vectorizer.fit_transform([' '.join(self.equipment_patterns[cat]) for cat in matches])
            text_vector = vectorizer.transform([combined_text])
            
            similarities = cosine_similarity(text_vector, category_vectors)[0]
            best_match_idx = np.argmax(similarities)
            return matches[best_match_idx]
        else:
            return 'Other'  # No clear equipment type found

# Create classifier instance
classifier = NLPEquipmentClassifier()

# Apply classification to the dataset
df['equipment_nlp'] = df.apply(
    lambda row: classifier.classify_equipment(str(row['EVT_DESC']), str(row['OBJ_DESC'])),
    axis=1
)

# Show distribution of new classifications
print("Equipment Classification Distribution:")
display(df['equipment_nlp'].value_counts())

# Compare with original classification
print("\nComparison with original classification:")
comparison_df = pd.crosstab(df['equipment'], df['equipment_nlp'])
display(comparison_df)

# Show some examples of each new equipment type
for equip_type in df['equipment_nlp'].unique():
    if equip_type == 'Other':
        continue
    print(f"\nExamples of {equip_type}:")
    sample = df[df['equipment_nlp'] == equip_type].sample(n=min(3, len(df[df['equipment_nlp'] == equip_type])))[['EVT_DESC', 'OBJ_DESC', 'equipment']]
    display(sample)

Equipment Classification Distribution:


equipment_nlp
Other                42434
Door                 22762
Drinking Fountain     2123
AHU                   1849
RTU                   1664
Fire Extinguisher     1613
Eye Wash Station      1418
Generator             1302
Elevator              1272
Trash Compactor       1063
Chiller                922
Escalator              836
Cooling Tower          550
Fire Panel             326
Boiler                 240
Water Heater           222
Window                 158
Minisplit              146
Baggage Carousel        56
Sprinkler               49
Baggage Belt            43
Electrical Panel        39
UPS                     28
Dumpster                15
PCA                      1
Name: count, dtype: int64


Comparison with original classification:


equipment_nlp,AHU,Baggage Belt,Baggage Carousel,Boiler,Chiller,Cooling Tower,Door,Drinking Fountain,Dumpster,Electrical Panel,Elevator,Escalator,Eye Wash Station,Fire Extinguisher,Fire Panel,Generator,Minisplit,Other,PCA,RTU,Sprinkler,Trash Compactor,UPS,Water Heater,Window
equipment,,,,,,,,,,,,,,,,,,,,,,,,,
Baggage Handling,63,11,18,0,2,6,3,0,0,0,0,0,2,0,1,107,0,313,0,10,1,0,0,0,0
Doors/Windows,12,6,9,14,77,2,17966,3,1,5,0,0,25,18,8,104,3,770,0,7,8,486,0,0,129
Electrical/Lighting,43,14,8,9,40,7,1536,1,2,11,0,0,2,1,9,80,8,3801,0,4,4,69,2,0,15
Elevator/Escalator,3,0,4,0,0,1,81,17,0,1,1272,836,1,2,2,4,1,136,0,0,0,35,1,6,2
Fire Safety,0,0,0,0,2,0,1,0,0,0,0,0,0,1168,270,3,0,48,0,0,17,0,0,0,0
Flooring,7,1,5,1,0,0,30,1,0,0,0,0,841,330,1,0,2,990,0,2,2,2,0,0,0
HVAC,158,1,0,2,72,32,617,0,0,1,0,0,114,0,3,5,4,3965,0,67,0,1,0,8,4
Other/Unclassified,1500,0,0,184,613,411,290,7,9,15,0,0,353,0,25,543,124,13972,1,204,0,228,11,0,0
Parking,1,0,0,1,1,0,66,0,1,2,0,0,35,82,1,425,1,1028,0,0,4,2,7,0,0



Examples of Door:


...

,EVT_DESC,OBJ_DESC,equipment
58052,ONT FAC Minisplit Unit Quarterly PM,OIAA Minisplit 212,Other/Unclassified
18379,Terminal 4 Mini Split PM 1316 Telecom Room,T4.F1-1316 / 4-149.0-TELECOM ROOM (ROOM 1DR),Other/Unclassified
45400,Re-key office 251 Admin Building A40 & M100 ke...,OIAA Minisplit 251,Other/Unclassified



Examples of Dumpster:


,EVT_DESC,OBJ_DESC,equipment
22601,GTC Car Rental pressure wash/clean near the du...,GTC.F1.-CHECK-IN AREA,Other/Unclassified
80469,Universal E-Waste container by train tracks. b...,WST.REC.YD.- WASTE & RECYCLE YARD EAST AVION ST.,Other/Unclassified
34932,MAINTENANCE YARD DUMPSTER IS FULL NEEDS TO BE ...,MTC.F1.-MAINTENANCE YARD,Other/Unclassified



Examples of PCA:


,EVT_DESC,OBJ_DESC,equipment
80475,Purchase Equipment for Jet Bridge Project for ...,T2.F2-JETWAY GATE 214,Other/Unclassified


In [20]:
# Enhanced NLP classification with context awareness
class EnhancedEquipmentClassifier(NLPEquipmentClassifier):
    def __init__(self):
        super().__init__()
        # Add contextual rules
        self.context_rules = {
            'location_excludes': {
                'room': r'(?i)(?:room|rm)',
                'area': r'(?i)(?:area|location|building)',
                'reference': r'(?i)(?:near|by|beside|close to)'
            },
            'maintenance_terms': {
                'repair': r'(?i)(?:repair|fix|replace)',
                'pm': r'(?i)(?:pm|preventive|maintenance)',
                'install': r'(?i)(?:install|setup|configure)'
            }
        }
        
    def is_equipment_reference(self, text, equipment_type):
        """Determine if the equipment mention is a reference or actual equipment"""
        text = text.lower()
        
        # Check if it's a maintenance activity
        is_maintenance = any(
            re.search(pattern, text)
            for patterns in self.context_rules['maintenance_terms'].values()
            for pattern in [patterns]
        )
        
        # Check if it's just a location reference
        is_location_ref = any(
            re.search(pattern, text)
            for patterns in self.context_rules['location_excludes'].values()
            for pattern in [patterns]
        ) and not is_maintenance
        
        return not is_location_ref or is_maintenance
    
    def classify_equipment(self, evt_desc, obj_desc):
        """Enhanced classification with context awareness"""
        combined_text = f"{evt_desc} {obj_desc}"
        matches = []
        
        for category, patterns in self.compiled_patterns.items():
            if any(pattern.search(combined_text) for pattern in patterns):
                if self.is_equipment_reference(combined_text, category):
                    matches.append(category)
        
        if len(matches) == 1:
            return matches[0]
        elif len(matches) > 1:
            # Use TF-IDF to find best match
            vectorizer = TfidfVectorizer()
            category_vectors = vectorizer.fit_transform([' '.join(self.equipment_patterns[cat]) for cat in matches])
            text_vector = vectorizer.transform([combined_text])
            similarities = cosine_similarity(text_vector, category_vectors)[0]
            return matches[np.argmax(similarities)]
        
        return 'Other'

# Apply enhanced classification
enhanced_classifier = EnhancedEquipmentClassifier()
df['equipment_enhanced'] = df.apply(
    lambda row: enhanced_classifier.classify_equipment(str(row['EVT_DESC']), str(row['OBJ_DESC'])),
    axis=1
)

# Show results of enhanced classification
print("Enhanced Equipment Classification Distribution:")
display(df['equipment_enhanced'].value_counts())

# Compare classifications
print("\nComparison of classification methods:")
comparison = pd.crosstab(df['equipment_nlp'], df['equipment_enhanced'])
display(comparison)

Enhanced Equipment Classification Distribution:


equipment_enhanced
Other                49695
Door                 18915
Drinking Fountain     1860
AHU                   1656
RTU                   1623
Fire Extinguisher     1583
Eye Wash Station      1374
Generator             1094
Elevator               836
Cooling Tower          526
Trash Compactor        494
Chiller                491
Fire Panel             225
Boiler                 156
Water Heater           133
Escalator              131
Minisplit              123
Window                  95
Sprinkler               28
Baggage Carousel        26
Electrical Panel        22
Baggage Belt            20
UPS                     15
Dumpster                 9
PCA                      1
Name: count, dtype: int64


Comparison of classification methods:


equipment_enhanced,AHU,Baggage Belt,Baggage Carousel,Boiler,Chiller,Cooling Tower,Door,Drinking Fountain,Dumpster,Electrical Panel,Elevator,Escalator,Eye Wash Station,Fire Extinguisher,Fire Panel,Generator,Minisplit,Other,PCA,RTU,Sprinkler,Trash Compactor,UPS,Water Heater,Window
equipment_nlp,,,,,,,,,,,,,,,,,,,,,,,,,
AHU,1656,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,193,0,0,0,0,0,0,0
Baggage Belt,0,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23,0,0,0,0,0,0,0
Baggage Carousel,0,0,26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,0,0,0,0
Boiler,0,0,0,156,0,0,0,0,0,0,0,0,0,0,0,0,0,84,0,0,0,0,0,0,0
Chiller,0,0,0,0,491,0,0,0,0,0,0,0,0,0,0,0,0,431,0,0,0,0,0,0,0
Cooling Tower,0,0,0,0,0,526,0,0,0,0,0,0,0,0,0,0,0,24,0,0,0,0,0,0,0
Door,0,0,0,0,0,0,18915,0,0,0,0,0,0,0,0,0,0,3847,0,0,0,0,0,0,0
Drinking Fountain,0,0,0,0,0,0,0,1860,0,0,0,0,0,0,0,0,0,263,0,0,0,0,0,0,0
Dumpster,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0


In [21]:
# Create an enhanced comparison DataFrame with all classifications
enhanced_comparison_df = df[[
    'EVT_DESC',
    'OBJ_DESC',
    'OBJ_CLASS',
    'OBJ_CATEGORY',
    'equipment',           # Original equipment classification
    'equipment_enhanced'   # New NLP-based classification
]].copy()

# Export to Excel with a new name to distinguish from the original
output_path = Path('../data/raw/equipment_classification_enhanced_results.xlsx')
enhanced_comparison_df.to_excel(output_path, index=False)

print(f"Exported enhanced comparison data to: {output_path}")

# Display a sample of the results where classifications differ
print("\nSample of records where classifications differ:")
different_classifications = enhanced_comparison_df[
    enhanced_comparison_df['equipment'] != enhanced_comparison_df['equipment_enhanced']
].sample(n=10)
display(different_classifications)

Exported enhanced comparison data to: ..\data\raw\equipment_classification_enhanced_results.xlsx

Sample of records where classifications differ:


,EVT_DESC,OBJ_DESC,OBJ_CLASS,OBJ_CATEGORY,equipment,equipment_enhanced
34352,ONT FAC Restroom Bi-Monthly PM,T4.F2-2065 / 4-205.0-MEN'S RESTROOM (ROOM 1DR),NaN,NaN,Plumbing,Other
26992,ONT FAC Door Quarterly PM,T2.F2- / KEYPAD -JET BRIDGE GATE 205 (DOOR 1DR),NaN,NaN,Doors/Windows,Door
49203,T4 by gate 410 remove & trash the desk and toa...,T4.F1.-RAMPSIDE,NaN,NaN,Structural,Other
11658,ONT FAC Door Quarterly PM,T2.F2-2272 / -ONT. BUSINESS CENTER (ROOM DR),NaN,NaN,Doors/Windows,Door
75023,ONT FAC Door Quarterly PM,T4.F2-2446 / 4-236.11-WOMEN'S RESTROOM (ROOM 1DR),NaN,NaN,Plumbing,Door
14468,ONT FAC Restroom Bi-Monthly PM,T2.F2-2310 / -WOMAN'S RESTROOM (ROOM DR),NaN,NaN,Plumbing,Other
76431,ONT FAC Door Quarterly PM,T2.F2-2454 / -ONT. TELEPHONE ROOM (ROOM 1DR),NaN,NaN,Doors/Windows,Door
7573,ONT FAC Fire Extinguisher Monthly PM,MAIN STORAGE FIRE EXTINGUISHERS,EQUIPMEN,NaN,Fire Safety,Fire Extinguisher
34573,Checked Leak In Taylor Mabry's Office,ADM.OFFCE.F1-ADMIN OFFICE,NaN,NaN,Other/Unclassified,Other
7825,gtc chiller clean condenser coil rest chiller ...,GTC CHILLER UNIT,FAC,HVAC,Other/Unclassified,Chiller


## Improving Classification Accuracy

Let's analyze misclassifications and add support for additional equipment types:

In [22]:
# First, let's analyze entries with "Exhaust Fan" or similar terms
exhaust_fan_mask = (
    df['EVT_DESC'].str.contains('exhaust fan|exhaust system', case=False, na=False) |
    df['OBJ_DESC'].str.contains('exhaust fan|exhaust system', case=False, na=False)
)

print("Sample of Exhaust Fan entries and their current classifications:")
exhaust_fan_entries = df[exhaust_fan_mask][['EVT_DESC', 'OBJ_DESC', 'equipment', 'equipment_enhanced']]
display(exhaust_fan_entries.head(10))

print("\nCurrent classification distribution for Exhaust Fan entries:")
display(exhaust_fan_entries['equipment_enhanced'].value_counts())

Sample of Exhaust Fan entries and their current classifications:


,EVT_DESC,OBJ_DESC,equipment,equipment_enhanced
788,Terminal 4 Carl's Jr.: Check Exhaust Fan 2.03.,T4.F2-2180 / -VACANT (SUITE DR),HVAC,Other
916,Terminal 2 lubricated bearings for roof exhaus...,EF 2.14,HVAC,Other
917,Terminal 4 lubricated bearings for roof exhaus...,EF 4.20,HVAC,Other
1032,ONT FAC Exhaust Fan Quarterly PM,EF 4.1,HVAC,Other
1033,ONT FAC Exhaust Fan Quarterly PM,EF. 4.9,HVAC,Other
1034,ONT FAC Exhaust Fan Quarterly PM,EF 4.20,HVAC,Other
1035,ONT FAC Exhaust Fan Quarterly PM,EF. 4.5,HVAC,Other
1036,ONT FAC Exhaust Fan Quarterly PM,EF 4.16,HVAC,Other
1037,ONT FAC Exhaust Fan Quarterly PM,EF 4.6,HVAC,Other
1038,ONT FAC Exhaust Fan Quarterly PM,EF. 4.15,HVAC,Other



Current classification distribution for Exhaust Fan entries:


equipment_enhanced
Other            1357
AHU                 4
Door                1
Cooling Tower       1
RTU                 1
Name: count, dtype: int64

In [23]:
# Improved classifier with hierarchical categories and confidence scoring
class ImprovedEquipmentClassifier(EnhancedEquipmentClassifier):
    def __init__(self):
        super().__init__()
        
        # Define equipment hierarchies
        self.equipment_hierarchies = {
            'HVAC': {
                'Air Handling': ['AHU', 'Exhaust Fan', 'Supply Fan', 'Return Fan'],
                'Cooling': ['Chiller', 'Cooling Tower', 'RTU', 'Minisplit'],
                'Heating': ['Boiler', 'Heat Pump'],
                'Air Distribution': ['VAV', 'Damper', 'Duct'],
                'Special Systems': ['PCA', 'Fan Coil']
            },
            'Fire Safety': {
                'Detection': ['Fire Panel', 'Smoke Detector', 'Heat Detector'],
                'Suppression': ['Fire Extinguisher', 'Sprinkler', 'Fire Hose']
            }
            # Add other hierarchies as needed
        }
        
        # Expand equipment patterns with more variations and components
        self.equipment_patterns.update({
            'Exhaust Fan': [
                'exhaust fan', 'exhaust system', 'exhaust unit',
                'fume hood', 'ventilation fan', 'vent fan'
            ],
            'Supply Fan': [
                'supply fan', 'supply air', 'supply unit',
                'air supply', 'ventilation supply'
            ],
            'Return Fan': [
                'return fan', 'return air', 'air return',
                'return unit'
            ],
            'Damper': [
                'damper', 'air damper', 'control damper',
                'isolation damper', 'fire damper'
            ],
            'Duct': [
                'duct', 'ductwork', 'air duct',
                'ventilation duct', 'duct system'
            ]
        })
        
        # Add component-specific patterns
        self.component_patterns = {
            'fan': [r'(?i)fan\s*(motor|belt|bearing|wheel|housing|assembly)?'],
            'filter': [r'(?i)(air|hepa|merv|pre|return)?\s*filter'],
            'motor': [r'(?i)(electric|drive)?\s*motor'],
            'belt': [r'(?i)(drive|fan)?\s*belt'],
            'bearing': [r'(?i)(fan|motor|shaft)?\s*bearing'],
            'coil': [r'(?i)(cooling|heating|condenser|dx)?\s*coil']
        }
        
        # Recompile patterns
        self.compiled_patterns = {
            category: [re.compile(r'\b' + term.replace(' ', r'\s+') + r'\b', re.IGNORECASE)
                      for term in terms]
            for category, terms in self.equipment_patterns.items()
        }
        
        # Add component patterns
        self.compiled_component_patterns = {
            component: [re.compile(pattern) for pattern in patterns]
            for component, patterns in self.component_patterns.items()
        }
    
    def get_confidence_score(self, text, category):
        """Calculate confidence score for a classification"""
        score = 0
        text = text.lower()
        
        # Direct equipment mention
        if any(pattern.search(text) for pattern in self.compiled_patterns[category]):
            score += 3
        
        # Component mentions
        component_matches = sum(
            1 for patterns in self.compiled_component_patterns.values()
            for pattern in patterns
            if pattern.search(text)
        )
        score += min(component_matches, 2)  # Cap component score at 2
        
        # Maintenance terms
        if any(re.search(pattern, text) for pattern in self.context_rules['maintenance_terms'].values()):
            score += 1
        
        # Location reference (negative score)
        if any(re.search(pattern, text) for pattern in self.context_rules['location_excludes'].values()):
            score -= 1
        
        return score / 6  # Normalize to 0-1

    def classify_with_confidence(self, evt_desc, obj_desc):
        """Classify equipment with confidence score"""
        combined_text = f"{evt_desc} {obj_desc}"
        matches = []
        
        # Check each equipment type
        for category, patterns in self.compiled_patterns.items():
            if any(pattern.search(combined_text) for pattern in patterns):
                confidence = self.get_confidence_score(combined_text, category)
                if confidence > 0.3:  # Minimum confidence threshold
                    matches.append((category, confidence))
        
        if matches:
            # Return highest confidence match
            matches.sort(key=lambda x: x[1], reverse=True)
            return matches[0]
        
        return ('Other', 0.0)

# Create improved classifier
improved_classifier = ImprovedEquipmentClassifier()

# Apply improved classification
results = []
for _, row in df.iterrows():
    equipment_type, confidence = improved_classifier.classify_with_confidence(
        str(row['EVT_DESC']), 
        str(row['OBJ_DESC'])
    )
    results.append({
        'equipment_improved': equipment_type,
        'confidence': confidence
    })

# Add results to dataframe
results_df = pd.DataFrame(results)
df['equipment_improved'] = results_df['equipment_improved']
df['classification_confidence'] = results_df['confidence']

# Show distribution of improved classifications
print("Improved Equipment Classification Distribution:")
display(df['equipment_improved'].value_counts())

# Show average confidence by equipment type
print("\nAverage confidence score by equipment type:")
avg_confidence = df.groupby('equipment_improved')['classification_confidence'].mean().sort_values(ascending=False)
display(avg_confidence)

# Compare with previous classification
print("\nSample of newly classified Exhaust Fan entries:")
exhaust_fan_entries = df[df['equipment_improved'] == 'Exhaust Fan'][
    ['EVT_DESC', 'OBJ_DESC', 'equipment', 'equipment_improved', 'classification_confidence']
].sort_values('classification_confidence', ascending=False)
display(exhaust_fan_entries.head(10))

Improved Equipment Classification Distribution:


equipment_improved
Other                41038
Door                 22555
Drinking Fountain     2123
AHU                   2037
RTU                   1671
Fire Extinguisher     1616
Exhaust Fan           1316
Elevator              1309
Generator             1188
Eye Wash Station      1163
Chiller               1067
Trash Compactor       1065
Escalator              847
Boiler                 583
Cooling Tower          524
Fire Panel             324
Water Heater           168
Minisplit              146
Window                 108
Sprinkler               50
Baggage Carousel        42
Supply Fan              38
Duct                    34
Baggage Belt            33
Electrical Panel        31
UPS                     28
Dumpster                15
Damper                   6
PCA                      4
Return Fan               2
Name: count, dtype: int64


Average confidence score by equipment type:


equipment_improved
Exhaust Fan          0.794580
Supply Fan           0.688596
Cooling Tower        0.663168
RTU                  0.630760
Fire Extinguisher    0.616337
AHU                  0.609884
Baggage Belt         0.590909
Minisplit            0.584475
PCA                  0.583333
Generator            0.563412
Eye Wash Station     0.546145
Door                 0.542873
Drinking Fountain    0.498822
Boiler               0.484848
Elevator             0.483575
Dumpster             0.477778
Window               0.462963
Duct                 0.455882
Fire Panel           0.450617
Water Heater         0.448413
Chiller              0.447829
Electrical Panel     0.446237
Sprinkler            0.430000
UPS                  0.428571
Baggage Carousel     0.428571
Trash Compactor      0.421596
Return Fan           0.416667
Damper               0.388889
Escalator            0.360685
Other                0.000000
Name: classification_confidence, dtype: float64


Sample of newly classified Exhaust Fan entries:


,EVT_DESC,OBJ_DESC,equipment,equipment_improved,classification_confidence
13672,Replace motor sheave,T4/ BHS EXHAUST FAN 2,HVAC,Exhaust Fan,1.000000
29725,T4 4.1 Exhaust Fan Replace Belt,EF 4.1,HVAC,Exhaust Fan,1.000000
61761,replace pulleys and belt,FIS EXHAUST FAN 2,HVAC,Exhaust Fan,1.000000
61762,replace motor on exhaust fan,FIS EXHAUST FAN 8,HVAC,Exhaust Fan,1.000000
61760,replace pulleys and belts,FIS EXHAUST FAN 4,HVAC,Exhaust Fan,1.000000
1059,ONT FAC Exhaust Fan Quarterly PM,EF 2.8,HVAC,Exhaust Fan,0.833333
54105,ONT FAC Exhaust Fan Quarterly PM,EF 4.20,HVAC,Exhaust Fan,0.833333
75677,ONT FAC Exhaust Fan Quarterly PM,FIRE STATION EF 3,HVAC,Exhaust Fan,0.833333
76931,ONT FAC Exhaust Fan Quarterly PM,FIRE STATION EF 3,HVAC,Exhaust Fan,0.833333
54112,ONT FAC Exhaust Fan Quarterly PM,EF. 4.4,HVAC,Exhaust Fan,0.833333


In [24]:
# First, let's analyze some "Other" classifications that should be HVAC
print("Analyzing potential missed HVAC equipment...")
other_hvac = df[
    (df['equipment_enhanced'] == 'Other') & 
    (
        df['EVT_DESC'].str.contains('exhaust fan|supply fan|return fan|fan motor', case=False, na=False) |
        df['OBJ_DESC'].str.contains('exhaust fan|supply fan|return fan|fan motor', case=False, na=False)
    )
][['EVT_DESC', 'OBJ_DESC', 'equipment_enhanced']]

print("Sample of missed fan-related equipment:")
display(other_hvac.head(10))

# Count common patterns in unclassified records
print("\nCommon patterns in unclassified records:")
pattern_counts = pd.Series(' '.join(other_hvac['EVT_DESC'].fillna('') + ' ' + other_hvac['OBJ_DESC'].fillna('')).lower().split()).value_counts()
display(pattern_counts.head(20))

Analyzing potential missed HVAC equipment...
Sample of missed fan-related equipment:
Sample of missed fan-related equipment:


,EVT_DESC,OBJ_DESC,equipment_enhanced
125,Maintenance Yard Replaced Condensor Fan Motor,MTC.YD.F1.-MAINTENANCE OFFICE,Other
443,HVAC TECH CHANGED OUT FILLTERS FOR ELECTRICAL ...,SUPPLY FAN 2.1,Other
788,Terminal 4 Carl's Jr.: Check Exhaust Fan 2.03.,T4.F2-2180 / -VACANT (SUITE DR),Other
829,Terminal 2 BHS supply fans in area and rplaced...,T2.F1-1324A / 2-135.0-BAGGAGE SERVICE (ROOM 1DR),Other
841,Terminal 4 replaced filter for supply fans for...,T4.F1-1020 / 4-114.5-ONT. FAN ROOM (ROOM 1DR),Other
916,Terminal 2 lubricated bearings for roof exhaus...,EF 2.14,Other
917,Terminal 4 lubricated bearings for roof exhaus...,EF 4.20,Other
1032,ONT FAC Exhaust Fan Quarterly PM,EF 4.1,Other
1033,ONT FAC Exhaust Fan Quarterly PM,EF. 4.9,Other
1034,ONT FAC Exhaust Fan Quarterly PM,EF 4.20,Other



Common patterns in unclassified records:


fan           1846
exhaust       1681
ont           1198
fac           1198
pm            1170
quarterly     1164
ef             635
room           222
fis            175
supply         175
switchgear     173
station        169
fire           167
bhs            162
2              147
ef.            146
1              128
3               94
t2/             79
t4/             73
Name: count, dtype: int64

In [25]:
class ImprovedEquipmentClassifier:
    def __init__(self):
        # Define equipment hierarchy
        self.equipment_hierarchy = {
            'HVAC': {
                'Air Handling': {
                    'patterns': [
                        r'(?i)(?:ahu|air\s*handler|air\s*handling\s*unit)',
                        r'(?i)(?:supply\s*fan|return\s*fan|exhaust\s*fan|ef[\s.#-]\d+)',
                        r'(?i)(?:fan\s*motor|fan\s*belt|fan\s*room)'
                    ],
                    'subtypes': ['AHU', 'Exhaust Fan', 'Supply Fan', 'Return Fan']
                },
                'Cooling': {
                    'patterns': [
                        r'(?i)(?:rtu|roof\s*top\s*unit)',
                        r'(?i)(?:chiller|cooling\s*system)',
                        r'(?i)(?:cooling\s*tower|condenser)',
                        r'(?i)(?:mini\s*split|split\s*system)'
                    ],
                    'subtypes': ['RTU', 'Chiller', 'Cooling Tower', 'Minisplit']
                },
                'Heating': {
                    'patterns': [
                        r'(?i)(?:boiler|heating\s*system)',
                        r'(?i)(?:heat\s*exchanger)',
                        r'(?i)(?:steam\s*system)'
                    ],
                    'subtypes': ['Boiler', 'Heat Exchanger']
                },
                'Air Distribution': {
                    'patterns': [
                        r'(?i)(?:vav|variable\s*air\s*volume)',
                        r'(?i)(?:damper|diffuser)',
                        r'(?i)(?:duct\s*work|ductwork)'
                    ],
                    'subtypes': ['VAV', 'Damper', 'Ductwork']
                },
                'Specialized': {
                    'patterns': [
                        r'(?i)(?:pca|pre[\s-]?conditioned\s*air)',
                        r'(?i)(?:mau|make[\s-]?up\s*air)',
                        r'(?i)(?:heat\s*pump)'
                    ],
                    'subtypes': ['PCA', 'MAU', 'Heat Pump']
                }
            }
        }
        
        # Compile all patterns
        self.compiled_patterns = {}
        for category, subcats in self.equipment_hierarchy.items():
            for subcat, info in subcats.items():
                for pattern in info['patterns']:
                    key = f"{category}_{subcat}"
                    if key not in self.compiled_patterns:
                        self.compiled_patterns[key] = []
                    self.compiled_patterns[key].append(re.compile(pattern))
        
    def get_confidence_score(self, text, patterns):
        """Calculate confidence score for a match"""
        score = 0
        text = text.lower()
        
        # Check for exact matches
        for pattern in patterns:
            matches = pattern.finditer(text)
            for match in matches:
                # Full matches get higher score
                score += 3 if match.group() == text.strip() else 1
                
                # Check for maintenance context
                if re.search(r'(?i)(?:maintenance|repair|replace|pm|check)', text):
                    score += 1
                    
                # Check for specific details
                if re.search(r'(?i)(?:number|#|model|serial|unit)', text):
                    score += 1
        
        return score / (5 * len(patterns))  # Normalize to 0-1
        
    def classify_equipment(self, evt_desc, obj_desc):
        """Classify equipment with hierarchy and confidence scoring"""
        combined_text = f"{evt_desc} {obj_desc}"
        results = []
        
        # Check each category and subcategory
        for category, subcats in self.equipment_hierarchy.items():
            for subcat, info in subcats.items():
                key = f"{category}_{subcat}"
                patterns = self.compiled_patterns[key]
                
                # Check if any pattern matches
                if any(pattern.search(combined_text) for pattern in patterns):
                    confidence = self.get_confidence_score(combined_text, patterns)
                    
                    # Determine specific subtype
                    subtype = None
                    for pattern in patterns:
                        match = pattern.search(combined_text)
                        if match:
                            matched_text = match.group().lower()
                            # Map to specific subtype based on the match
                            if 'exhaust' in matched_text or 'ef' in matched_text:
                                subtype = 'Exhaust Fan'
                            elif 'supply' in matched_text:
                                subtype = 'Supply Fan'
                            elif 'return' in matched_text:
                                subtype = 'Return Fan'
                            # Add more specific mappings as needed
                    
                    results.append({
                        'category': category,
                        'subcategory': subcat,
                        'subtype': subtype or info['subtypes'][0],
                        'confidence': confidence
                    })
        
        if results:
            # Sort by confidence and return the highest confidence result
            best_match = max(results, key=lambda x: x['confidence'])
            return best_match['subtype']
        
        return 'Other'

# Create and apply the improved classifier
improved_classifier = ImprovedEquipmentClassifier()
df['equipment_improved'] = df.apply(
    lambda row: improved_classifier.classify_equipment(str(row['EVT_DESC']), str(row['OBJ_DESC'])),
    axis=1
)

# Show distribution of new classifications
print("Improved Equipment Classification Distribution:")
display(df['equipment_improved'].value_counts())

# Compare with previous classification
print("\nComparison with previous classification:")
comparison = pd.crosstab(df['equipment_enhanced'], df['equipment_improved'])
display(comparison)

# Export results with new classification
enhanced_comparison_df['equipment_improved'] = df['equipment_improved']
output_path = Path('../data/raw/equipment_classification_improved_results.xlsx')
enhanced_comparison_df.to_excel(output_path, index=False)

print(f"\nExported improved comparison data to: {output_path}")

# Show sample of newly classified fan equipment
print("\nSample of newly classified fan equipment:")
fan_equipment = df[df['equipment_improved'].isin(['Exhaust Fan', 'Supply Fan', 'Return Fan'])].sample(10)
display(fan_equipment[['EVT_DESC', 'OBJ_DESC', 'equipment_improved']])

Improved Equipment Classification Distribution:


equipment_improved
Other          72340
RTU             3709
AHU             2886
Exhaust Fan     1414
Boiler           610
VAV               64
Supply Fan        63
PCA               25
Return Fan        20
Name: count, dtype: int64


Comparison with previous classification:


equipment_improved,AHU,Boiler,Exhaust Fan,Other,PCA,RTU,Return Fan,Supply Fan,VAV
equipment_enhanced,,,,,,,,,
AHU,1621,1,4,0,0,1,14,15,0
Baggage Belt,0,0,0,20,0,0,0,0,0
Baggage Carousel,0,0,0,26,0,0,0,0,0
Boiler,1,154,0,0,0,0,0,1,0
Chiller,26,6,0,2,1,455,0,0,1
Cooling Tower,12,0,5,0,0,507,0,1,1
Door,314,102,2,18393,5,95,0,0,4
Drinking Fountain,0,0,0,1860,0,0,0,0,0
Dumpster,0,0,0,9,0,0,0,0,0



Exported improved comparison data to: ..\data\raw\equipment_classification_improved_results.xlsx

Sample of newly classified fan equipment:


,EVT_DESC,OBJ_DESC,equipment_improved
31735,ONT FAC Exhaust Fan Quarterly PM,T2.F1-1444 SWITCHGEAR ROOM EXHAUST FAN,Exhaust Fan
22785,ONT FAC Exhaust Fan Quarterly PM,EF 2.6,Exhaust Fan
6762,ONT FAC Exhaust Fan Quarterly PM,EF 2.6,Exhaust Fan
60533,pm on make-up area exhaust fan 1,MAKE-UP AREA (BAG ROOM),Exhaust Fan
61231,ONT FAC Exhaust Fan Quarterly PM,FIS EXHAUST FAN 6,Exhaust Fan
55068,ONT FAC Exhaust Fan Quarterly PM,FIS EXHAUST FAN 6,Exhaust Fan
43963,ONT FAC Exhaust Fan Quarterly PM,EF 2.14,Exhaust Fan
81026,T2 Room 1596 Replace Pulleys & Belts For Exhau...,T2.F1-1596 / 3-117.0-SWITCHGEAR ROOM (ROOM 2DR),Exhaust Fan
35158,ONT FAC Exhaust Fan Quarterly PM,T4/ BHS SUPPLY FAN 1,Exhaust Fan
8173,t2 adjust belt and calibrate sheve for supply ...,AHU 2.14,Supply Fan
